In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Production & Terabyte Scale RAG Pipeline Using BigFrames

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fnotebook_template.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/notebook_template.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery Studio logo"><br> Open in BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>


<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>
```



| | |
|-|-|
| Authors | [Lorenzo Spataro](https://github.com/lspataroG)  [Elia Secchi](https://github.com/eliasecchig)  |


# Overview

This notebook demonstrates how to use BigFrames and LangChain to build a RAG (Retrieval Augmented Generation) pipeline using Vertex AI. You will learn how to:
- Load data from BigQuery into BigFrames
- Create embeddings using Vertex AI models
- Build a vector store using BigQuery
- Create a RAG pipeline using LangChain
- Query your data using natural language

## What is BigQuery DataFrames?
BigQuery DataFrames also called BigFrames lets you process data in BigQuery using familiar Python APIs like pandas and scikit-learn. It works by converting Python code into optimized SQL that runs directly in BigQuery.
Key benefits:
- Process terabytes of data using Python without moving it out of BigQuery
- Train ML models directly in BigQuery using scikit-learn syntax
- 750+ pandas/scikit-learn APIs available through SQL conversion
- Lazy evaluation for better performance
- Custom Python functions run as BigQuery remote functions
- Vertex AI integration for Gemini model access

The following diagram describes the workflow of BigQuery DataFrames:
![BigQuery DataFrames Workflow](https://cloud.google.com/static/bigquery/images/dataframes-workflow.png)




# Get started

## Install Vertex AI SDK and other required packages


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform  "bigframes" langchain markdownify swifter "langchain-google-community[featurestore]" langchain-google-vertexai


## Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. In Colab or Colab Enterprise, you might see an error message that says "Your session crashed for an unknown reason." This is expected. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
# Use the environment variable if the user doesn't provide Project ID.
import datetime
import os

import vertexai
from google.cloud import bigquery
import bigframes.ml.llm as llm
import bigframes.pandas as bpd

PROJECT_ID = ""  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

# GOOGLE_CLOUD_REGION must be in a US region because the source dataset is in US
LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

vertexai.init(project=PROJECT_ID, location=LOCATION)

bq_client = bigquery.Client(project=PROJECT_ID, location="US")
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = "US"

### Import libraries

In [ ]:
# Standard library imports
import json
from datetime import datetime, timedelta

# Third-party imports
import swifter
from google.cloud import bigquery
from langchain_google_community import BigQueryVectorStore
from langchain_google_vertexai import VertexAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from markdownify import markdownify

### Variables definition

As we're building a pipeline intended to run regularly on a schedule, we're going to set up some time-dependent variables:

*   `RUN_DATE`: The date the process runs
*   `IS_INCREMENTAL`: If `True` only query recent data; otherwise, query the whole dataset
*   `LOOK_BACK_DAYS`: If `IS_INCREMENTAL=True`, defines how many days in the past to query




In [ ]:
IS_INCREMENTAL = True  # Flag to enable incremental processing
RUN_DATE = datetime.strptime("2022-09-26", '%Y-%m-%d').date()  # Set as the last date of the dataset for demonstration purposes (ie. there is no data after that)
LOOK_BACK_DAYS = 1  # Number of days to look back from RUN_DATE
START_DATE = str(RUN_DATE - timedelta(days=LOOK_BACK_DAYS))  # Start date for data processing window
END_DATE = str(RUN_DATE)  # End date for data processing window

## 1. Data Loading and Initial Analysis
This section retrieves and examines StackOverflow Python Q&A data from the public BigQuery table `production-ai-template.stackoverflow_qa.stackoverflow_python_questions_and_answers`. The data comes from the official [Stack Overflow public dataset](https://console.cloud.google.com/marketplace/product/stack-exchange/stack-overflow) and contains a sample of Python-related questions and their corresponding answers.


In [ ]:
query = f"""
    SELECT
        creation_date,
        last_edit_date,
        question_id,
        question_title,
        question_body AS question_text,
        answers
    FROM `production-ai-template.stackoverflow_qa.stackoverflow_python_questions_and_answers`
    WHERE TRUE
        # If IS_INCREMENTAL is True, filter records between START_DATE and END_DATE
        # Otherwise, include all records without date filtering
        {f'AND TIMESTAMP_TRUNC(creation_date, DAY) BETWEEN TIMESTAMP("{START_DATE}") AND TIMESTAMP("{END_DATE}")' if IS_INCREMENTAL else ''}
"""
df = bpd.read_gbq(query)
df.head(2)

## 2. Data Cleaning and Markdown Conversion
In this step, we clean the raw data by converting HTML content to Markdown format for better readability and processing.
We transform both questions and answers from HTML to Markdown, structure the content with proper headings,
and combine them into a unified text format that will be easier to work with in subsequent steps.

> Note: In this case, we are leveraging BigFrames' capability to pull data into memory, where the processing happens. This allows us to efficiently transform and clean the data using pandas-like operations. Later, we will demonstrate how to scale this data processing using Remote Functions.

In [ ]:
def convert_html_to_markdown(html: str) -> str:
    """Convert HTML into Markdown for easier parsing and rendering after LLM response."""
    return markdownify(html).strip()

def create_answers_markdown(answers: list[dict]) -> str:
    """Convert each answer's HTML to markdown and concatenate into a single markdown text."""
    answers_md = ""
    for index, answer_record in enumerate(answers):
        answers_md += f"\n\n## Answer {index + 1}:\n" # Answer number is H2 heading size
        answers_md += convert_html_to_markdown(answer_record["body"])
    return answers_md

In [ ]:
# Sort, deduplicate and reset index in one operation
df = df.sort_values("last_edit_date", ascending=False).drop_duplicates("question_id").reset_index(drop=True)

# Create markdown fields efficiently
df["question_title_md"] = "# " + df["question_title"] + "\n"  # Title is H1 heading size
df["question_text_md"] = df["question_text"].to_pandas().apply(convert_html_to_markdown)
df["answers_md"] = df["answers"].to_pandas().apply(create_answers_markdown)

# Load data back in BigQuery
df["full_text_md"] = df["question_title_md"] + df["question_text_md"] + df["answers_md"]

In [ ]:
# Select final columns for the cleaned dataset
final_cols = ["last_edit_date", "question_id", "question_text", "full_text_md"]
df = df[final_cols]
df.head(2)

## 2. Text Chunking
The text data is in Markdown format, requiring a thoughtful chunking approach. While we currently use a basic character-based splitter, production systems typically employ more sophisticated techniques:

- Preserve semantic units like paragraphs and sections
- Maintain markdown structure and hierarchy
- Keep related content together (e.g., questions with their answers)
- Use overlapping chunks to maintain context across boundaries
- Consider special markdown elements like code blocks and lists

This helps ensure chunks remain coherent and meaningful for downstream tasks.


In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=20,
    length_function=len,
)

Apply text chunking to each document locally using pandas and swifter


In [ ]:
df["text_chunk"] = df["full_text_md"].to_pandas().astype(object).swifter.apply(text_splitter.split_text)
df.head(2)

In [ ]:
# compute the sequential index of a chunk within the list of chunks for each question
chunk_ids = [str(idx) for text_chunk in df["text_chunk"] for idx in range(len(text_chunk))]
# Explode the chunk list so that we get a row per chunk
df = df.explode("text_chunk").reset_index(drop=True)
# assigning the chunk_id as question_id + sequential index of the chunk
df["chunk_id"] = df["question_id"].astype("string") + "__" + chunk_ids

In [ ]:
df.head()

## 3. Embedding

To generate embeddings, we leverage the seamless integration between BigFrames, BigQuery, and Vertex AI.
This integration allows us to efficiently generate embeddings through Vertex AI's batch scoring process.
The text-embedding-005 model converts each text chunk into a high-dimensional vector representation,
enabling semantic search and similarity analysis.


In [ ]:
# Initialize the embedding model
embedder = llm.TextEmbeddingGenerator(model_name="text-embedding-005")

# Generate embeddings
embeddings_df = embedder.predict(df["text_chunk"])
df = df.assign(
    embedding_result=embeddings_df["ml_generate_embedding_result"],
    embedding_statistics=embeddings_df["ml_generate_embedding_statistics"],
    embedding_status=embeddings_df["ml_generate_embedding_status"]
)
current_timestamp = datetime.now()
df["creation_timestamp"] = current_timestamp

df.head()

We can now notice 4 new columns added to our DataFrame!

## Saving results

We are now ready to save the results of the processing to a BigQuery table, for consumption by the different Vector DBs we might want to use.
The incremental writing strategy allows us to efficiently update our embeddings table by:
1. Only processing new/modified questions since last run (controlled by `IS_INCREMENTAL` flag)
2. Appending new embeddings to the existing table when `IS_INCREMENTAL=True`
3. Replacing the entire table when `IS_INCREMENTAL=False`

Since we may end up with duplicate entries when doing incremental updates
(e.g. if a question was modified multiple times), we'll need to deduplicate
the table afterwards to keep only the latest version of each question.
The deduplication will be done based on question_id, keeping the row with the most recent `creation_timestamp`.


In [ ]:
DESTINATION_DATASET_ID = "stackoverflow_data"
DESTINATION_TABLE_ID = "incremental_questions_embeddings"
PARTITION_DATE_COLUMN = "creation_timestamp"

### If it doesn't exist, let's create an empty table with partitioning and the right schema

In [ ]:
def create_table_if_not_exist(df, project_id, dataset_id, table_id, partition_column, location="US"):
    table_schema = bq_client.get_table(df.head(0).to_gbq()).schema

    # Create table schema with partitioning
    table = bigquery.Table(f"{project_id}.{dataset_id}.{table_id}", schema=table_schema)
    table.time_partitioning = bigquery.TimePartitioning(
        type_=bigquery.TimePartitioningType.DAY,
        field=partition_column
    )

    dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
    dataset.location = location
    bq_client.create_dataset(dataset, exists_ok=True)
    table = bq_client.create_table(table=table, exists_ok=True)

create_table_if_not_exist(
    df=df,
    project_id=PROJECT_ID,
    dataset_id=DESTINATION_DATASET_ID,
    table_id=DESTINATION_TABLE_ID,
    partition_column=PARTITION_DATE_COLUMN
)

In [ ]:
# If IS_INCREMENTAL is True, append new data to existing table
# If IS_INCREMENTAL is False, replace entire table with new data
if_exists_mode = "append" if IS_INCREMENTAL else "replace"

incremental_table_id = df.to_gbq(
    destination_table=f"{DESTINATION_DATASET_ID}.{DESTINATION_TABLE_ID}",
    if_exists=if_exists_mode
)

## Create a new Dedup table (Optional)

If necessary, we can create a deduplication table to address duplicate questions that may appear in the dataset across different dates.

In [ ]:
df_questions = bpd.read_gbq(f"{DESTINATION_DATASET_ID}.{DESTINATION_TABLE_ID}", use_cache=False)
max_date_df = df_questions.groupby("question_id")["creation_timestamp"].max().reset_index()
df_questions_dedup = max_date_df.merge(df_questions, how="inner", on=["question_id", "creation_timestamp"])

In [ ]:
DESTINATION_DEDUPED_QUESTIONS_TABLE_ID = "questions_embeddings"
create_table_if_not_exist(
    df=df_questions_dedup,
    project_id=PROJECT_ID,
    dataset_id=DESTINATION_DATASET_ID,
    table_id=DESTINATION_DEDUPED_QUESTIONS_TABLE_ID,
    partition_column=PARTITION_DATE_COLUMN
)

deduped_table_id = df_questions_dedup.to_gbq(
    destination_table=f"{DESTINATION_DATASET_ID}.{DESTINATION_DEDUPED_QUESTIONS_TABLE_ID}",
    if_exists="replace",
)

## Testing retrieval

In [ ]:
embedding_model = VertexAIEmbeddings(
    model_name="text-embedding-005", project=PROJECT_ID
)
bq_store = BigQueryVectorStore(
    project_id=PROJECT_ID,
    location="US",
    dataset_name=DESTINATION_DATASET_ID,
    table_name=DESTINATION_DEDUPED_QUESTIONS_TABLE_ID,
    embedding=embedding_model,
    embedding_field="embedding_result",
    content_field="text_chunk"
)


In [ ]:
bq_store.similarity_search("how to pull csv files?")

## Scaling to Terabytes: BigFrame Remote Functions

Sometimes data is too large to run local process when running custom python functions.
In fact, everytime we convert a series or a dataframe to pandas using `to_pandas()`, the data is loaded into memory.

To be able to run large datasets processes remotely, we can define remote <b>UDF functions</b>. Let's see an example of that.

In [ ]:
import bigframes.pandas
import bigframes.bigquery as bbq
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
)

# Create UDF for chunking
# Behind the scenes, BigFrames will automatically create a connection for you but you can also create a dedicated connection.
# See here: https://cloud.google.com/bigquery/docs/remote-functions#create_a_connection.
@bigframes.pandas.remote_function(packages=["langchain"], reuse=True)
def chunk_text_udf(text: str) -> str:
    return json.dumps([chunk.page_content for chunk in text_splitter.create_documents([text])])

We are using the UDF to chunk the data and return a list of text chunks. Since BigFrames UDFs expect in input and in output simple types, we are going to convert the list of chunks to a json string inside the UDF.

In [ ]:
df_udf = bpd.read_gbq(f"{DESTINATION_DATASET_ID}.{DESTINATION_DEDUPED_QUESTIONS_TABLE_ID}", use_cache=False)
df_udf["question_text"] = df_udf["question_text"].apply(chunk_text_udf)

As we can see, the data type is now string

In [ ]:
first_row_question = df_udf["question_text"].iloc[0]
print(type(first_row_question))
first_row_question

To fix that, we can now use the BigFrames BQ `json_extract_string_array` method to convert the json string back to a list of string

In [ ]:
df_udf["question_text"] = bbq.json_extract_string_array(df_udf["question_text"])

first_row_question = df_udf["question_text"].iloc[0]
print(type(first_row_question))
first_row_question

Let's print out the result. The remote function will be only executed now we ask to print it following a lazy execution.

## Cleaning up

Clean up resources created in this notebook.

In [ ]:
dataset = f"{PROJECT_ID}.{DESTINATION_DATASET_ID}"
dataset_object = bigquery.Dataset(dataset)
bq_client.delete_dataset(dataset_object, delete_contents=True, not_found_ok=True)

## Conclusion

This notebook showcased the power of **BigFrames** for building production-ready RAG pipelines on Google Cloud. We leveraged BigFrames' seamless integration with BigQuery and Vertex AI to efficiently process and embed large text datasets.

Key takeaways highlighting BigFrames' capabilities include:

*   **Scalable Data Processing:**  BigFrames allowed us to manipulate BigQuery data using familiar pandas-like syntax, whether processing in memory or through scalable remote functions for terabyte-scale datasets.
*   **Simplified Embedding Generation:** BigFrames made it easy to generate embeddings with Vertex AI's embedding models directly within our data pipeline.
*   **Efficient Data Management:** We used BigFrames to manage our embeddings in BigQuery, implementing incremental updates and deduplication for optimal performance.